### Задача (варіант 1)

**Ієрархія:** p = 3, повна, m = (4, 3, 3)

<img src="images/lab_2_hierarchy.png" width=256>

**Метод розрахунку глобальних ваг:** дистрибутивний, ГВБВПА

*Методи аналізу альтернатив рішень на основі ієрархічної моделі критеріїв (МАІ, analytic hierarchy process, AHP)* складаються з наступних чотирьох загальних етапів:

1. Побудова ієрархічної моделі критеріїв, цілей та інших факторів, які впливають на головну ціль прийняття рішення; побудова множини альтернативних варіантів рішень.

2. Отримання суджень експертів щодо парних порівнянь елементів одного рівня ієрархії відносно спільного елементу вищого рівня. Парні порівняння проводяться у вибраній шкалі і за результатами будуються матриці парних порівнянь (МПП), які є обернено симетричними.

3. Математична обробка суджень експертів:
   * розрахунок локальних ваг елементів кожного рівня ієрархії відповідно до батьківських елементів вищого рівня на основі МПП; побудова локальних ранжувань;
   * аналіз узгодженості експертних оцінок;
   * розрахунок глобальних ваг елементів ієрархії відносно головної цілі прийняття рішення, використовуючи методи агрегування; побудова ранжування на основі глобальних ваг.

4. Аналіз чутливості отриманих ранжувань.

Постановка задачі:

**Дано:** 
 * $A = \{A_i | i = 1, \dots, N \}$ - множина альтернативних варіантів рішень;
 * $C = \{C_j | j = 1, \dots, M \}$ - множина критеріїв оцінювання альтернатив;
 * $a_{ij}$ - ненормована вага льтернативи $A_i$ за критерієм $C_j$;
 * $w_j^C$ - вага критерію $C_j, \sum_{j=1}^{M} w_j^C = 1$

**Потрібно:**
 * знайти глобальні ваги $w_i^{глоб}$ альтернатив $A_i, i = 1, \dots, N$

In [1]:
import numpy as np
# put some fixed seed for reproducibility of the results
np.random.seed(42)

In [2]:
# we work with fully connected hierarcy, its architecture is defined by number of nodes per layer
# All layers but the last 'layers[:-1]' - creiterions
layers = [1, 4, 3, 3]
# last layer - 'layers[-1]' - alternatives
num_alternatives = layers[-1]
# number of layers
p = len(layers) - 1

Функція для генерування коректної матриці парних порівнянь для мультиплікативної групи

In [3]:
def generate_matched_matrix(dim, low=1, high=10):
    """
    Generates matched multiplicational matrix mmp i.e. \forall i \in [0, n-1]: mpp[i][i] = 1;
    \forall i, j i \neq j: mpp[i][j] = 1 / mpp[j][i]
    Returns 2D matched matrix of shape (dim, dim)
    """
    mmp = np.zeros((dim, dim))
    temp = np.random.randint(low, high, size=dim).astype(float)
    powers = np.random.choice(3, size=dim, p=[0.5, 0, 0.5]) - 1
    mmp[0, :] = np.power(temp, powers)
    j = 0
    for i in range(1, dim):
        mmp[i, j] = 1 / mmp[j, i]
    for i in range(1, dim):
        for j in range(1, dim):
            mmp[i, j] = mmp[i, 0] * mmp[0, j]
    mmp[0][0] = 1
    return mmp

# Data generation

Генеруємо матриці парних порівнянь.

Наша ієрархія - (4, 3, 3), фактично працюємо з ієрархією (1, 4, 3, 3). Тобто експертами формуються наступні матриці парних порівнянь
* 4x4x1 - парні порівняння 4 критеріїв відносно 1 критерію (мети)
* 3x3x4 - парні порівняння 3 підкритеріїв відносно 4 критеріїв
* 3х3х3 - парні порівняння 3 альтернатив відносно 3 підкритеріїв

Отримані дані об'єднаємо в список.

In [4]:
# generally speaking, number of nodes can vary from layer to layer, so further we use nested lists,
# not a np.array

D = []
for idx_curr in range(1, len(layers)):
    d = np.empty((layers[idx_curr], layers[idx_curr], layers[idx_curr - 1]))
    for i in range(layers[idx_curr - 1]):
        mpp = generate_matched_matrix(layers[idx_curr])
        d[:, :, i] = mpp
    D.append(d)

# display shapes and values of generated MPPs
for mpp in D:
    print(mpp.shape)
    print(mpp)
    print("*" * 80)

(4, 4, 1)
[[[1.00e+00]
  [2.50e-01]
  [1.25e-01]
  [5.00e+00]]

 [[4.00e+00]
  [1.00e+00]
  [5.00e-01]
  [2.00e+01]]

 [[8.00e+00]
  [2.00e+00]
  [1.00e+00]
  [4.00e+01]]

 [[2.00e-01]
  [5.00e-02]
  [2.50e-02]
  [1.00e+00]]]
********************************************************************************
(3, 3, 4)
[[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [1.25000000e-01 2.00000000e+00 1.00000000e+00 3.33333333e-01]
  [8.00000000e+00 2.00000000e-01 3.33333333e-01 7.00000000e+00]]

 [[8.00000000e+00 5.00000000e-01 1.00000000e+00 3.00000000e+00]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
  [6.40000000e+01 1.00000000e-01 3.33333333e-01 2.10000000e+01]]

 [[1.25000000e-01 5.00000000e+00 3.00000000e+00 1.42857143e-01]
  [1.56250000e-02 1.00000000e+01 3.00000000e+00 4.76190476e-02]
  [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]]]
********************************************************************************
(3, 3, 3)
[[[1.00000

Обраховуємо локальні ваги використовуючи підхід EM з попередньої лабораторної роботи.

**ЕМ метод** Згідно цього метода вектором ваг є власний вектор МПП, що відповідає її найбільшому власному числу. Тобто кожна МПП буде замінена вектором тієї ж розмірності, що і МПП. Отже на виході функції мусимо отримати список тензорів наступної розмірності:

* 4х1
* 3х1
* 3х1

In [18]:
# # # generate unnormed local weights of alternatives with respect to all criterions
# # # rows are corresponding to alternatives, columns - to criterions
# # v = np.random.randn(n, m)
# from numpy import linalg
# def calculate_local(D):
#     """
#     D: rank 3 ndarray of shape (m, n, n). m - number of criterions, n - number of alternatives
#     Returns (m, n) ndarray where each row represents local weights according to i-th criterion (i in [0, m-1])
#     """
#     m, n = D.shape[0], D.shape[1]
#     res = np.empty((m, n))
#     for idx in range(m):
#         w, v = linalg.eig(D[idx, :, :])
#         v_max = np.real(v[np.argmax(w)])
#         res[idx, :] = v_max
#     return res
# v = calculate_local(D)
# v

def _get_max_eigenval_eigenvec(matrix):
    """
    Requires special matrices for the input.
    mat: 2D numpy array
    return: eigenvector corresponding to the max eigenvalue
    """
    w, v = np.linalg.eig(matrix)
    w_max = np.real(np.max(w))
    v_max = np.real(v[np.argmax(w)])
    return v_max

def calculate_local(D):
    """
    D: list of rank 3 tensors - MPPs - of shape (n_curr, n_curr, n_prev)
    return: 
    """
    # create list which we will return
    res = []
    for i in range(len(D)):
        res.append(np.empty(D[i].shape[1:]))
#         print(res[i].shape)
    # for each set of MPP for current layer
    for idx_mpps_group in range(len(D)):
#         print(D[idx_mpps_group].shape)
        # we iterate over the last, 2nd dimension
        for idx_mpp in range(D[idx_mpps_group].shape[-1]):
            mpp = D[idx_mpps_group][:, :, idx_mpp]
#             print(mpp.shape)
            v_max = _get_max_eigenval_eigenvec(mpp)
            res[idx_mpps_group][:, idx_mpp] = v_max
    return res
local_weights = calculate_local(D)
local_weights

[array([[ 0.4239039 ],
        [-0.44433475],
        [ 0.13148054],
        [ 0.13148054]]),
 array([[ 1.24019829e-01,  1.95180015e-01,  3.01511345e-01,
         -3.15905578e-01],
        [-1.15093573e-17, -8.72121400e-02,  5.31486451e-01,
          1.71192270e-01],
        [ 9.78628409e-02, -8.72121400e-02, -5.95235988e-02,
         -2.57399587e-01]]),
 array([[ 0.1104046 ,  0.6882472 ,  0.95770426],
        [ 0.0982167 ,  0.88598299,  0.89735882],
        [ 0.0982167 , -0.27204132, -0.93110321]])]

In [21]:
lst_1 = [0.18148095867689784, 0.5133056661466803, 0.30521337517642183];
print(sum(lst_1))
lst_2 = [0.38356076046023996, 0.38356076046023996, 0.23287847907952017];
print(sum(lst_2))
lst_3 = [0.32087543988563444,
  0.1482822537233742,
  0.29837197692374073,
  0.23247032946725066]
print(sum(lst_3))

1.0
1.0
1.0


# Methods for finding global normed weights of alternatives

In [ ]:
# distributive synthesis approach
def distributed_synthesis(w_crit, v):
    """
    Computes global weights
    """
    n, m = v.shape
    w_glob = np.empty(n)
    for i in range(n):
        # compute 'w_globe' components one by one
        r = v[i, :] / np.sum(v[:, i])
        r = r.reshape(m, 1)
        print("w_crit.shape =", w_crit.shape)
        print("r.shape =", r.shape)
        w_glob[i] = np.dot(w_crit.T, r)
    return w_glob

In [ ]:
print(distributed_synthesis(w_crit, v).shape)
print(distributed_synthesis(w_crit, v))

In [ ]:
w_glob = distributed_synthesis(w_crit, v)
print(np.sum(w_glob))